<a href="https://colab.research.google.com/github/LukmaanViscomi/AI-Deep-Learning/blob/main/Baseline_Controlled_V4_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Section 1: Setup and Dependencies

In [2]:
### Section 1: Setup and Dependencies
!apt-get install p7zip-full # Install 7-Zip
!pip install patool # Install the patool library which provides the patoolib module
import zipfile
import os
import patoolib # Now you can import patoolib

# Path to the uploaded zip file
zip_file_path = 'dataset2 (1).zip'
extracted_folder_path = './dataset2'  # Use a relative path for the extraction directory

# Extract the zip file using patool
patoolib.extract_archive(zip_file_path, outdir=extracted_folder_path)

# List the contents of the extracted folder
extracted_files = os.listdir(extracted_folder_path)
print(extracted_files)



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


INFO patool: Extracting dataset2 (1).zip ...
INFO:patool:Extracting dataset2 (1).zip ...
INFO patool: ... creating output directory `./dataset2'.
INFO:patool:... creating output directory `./dataset2'.
INFO patool: running /usr/bin/7z x -o./dataset2 -- "dataset2 (1).zip"
INFO:patool:running /usr/bin/7z x -o./dataset2 -- "dataset2 (1).zip"
INFO patool:     with input=''
INFO:patool:    with input=''
INFO patool: ... dataset2 (1).zip extracted to `./dataset2'.
INFO:patool:... dataset2 (1).zip extracted to `./dataset2'.


['triple_mnist']


### Section 2 : Inspecting the Original Dataset

In [3]:
### Section 2 : Inspecting the Original Dataset
import os

# Path to the triple_mnist directory
base_dir = 'dataset2/triple_mnist'  # Replace with the actual path to the triple_mnist folder

# Folders to check
folders = ['train', 'test', 'val']

# Function to count classes and images in a given directory
def count_classes_and_images(directory):
    class_count = 0
    total_images = 0

    for class_name in os.listdir(directory):
        class_dir = os.path.join(directory, class_name)
        if os.path.isdir(class_dir):
            class_count += 1
            image_count = len(os.listdir(class_dir))
            total_images += image_count

    return class_count, total_images

# Iterate through each folder (train, test, val) and print the summary
for folder in folders:
    folder_path = os.path.join(base_dir, folder)
    class_count, total_images = count_classes_and_images(folder_path)
    print(f"{folder.capitalize()} folder: {class_count} classes, {total_images} images")


Train folder: 640 classes, 64000 images
Test folder: 200 classes, 20000 images
Val folder: 160 classes, 16000 images


### Section 3:  Paths to Your Full Dataset

In [4]:
### Section 3: Paths to Your Full Dataset
from pathlib import Path

# Paths to original directories
full_train_dir = Path('dataset2/triple_mnist/train')
full_val_dir = Path('dataset2/triple_mnist/val')
full_test_dir = Path('dataset2/triple_mnist/test')

# Paths to subset directories
subset_base_dir = Path('subset/triple_mnist')
subset_train_dir = subset_base_dir / 'train'
subset_val_dir = subset_base_dir / 'val'
subset_test_dir = subset_base_dir / 'test'

# Ensure subset directories exist
subset_train_dir.mkdir(parents=True, exist_ok=True)
subset_val_dir.mkdir(parents=True, exist_ok=True)
subset_test_dir.mkdir(parents=True, exist_ok=True)

print("Paths set for full and subset datasets.")



Paths set for full and subset datasets.


### Section 4: Create a Small Subset of Your Data

In [5]:
### Section 4: Create a Small Subset of Your Data
import shutil
import random

# Function to create a subset of the dataset
def create_subset(original_dir, subset_dir, classes, num_images_per_class):
    if not os.path.exists(subset_dir):
        os.makedirs(subset_dir)
    for class_name in classes:
        class_dir = os.path.join(original_dir, class_name)
        subset_class_dir = os.path.join(subset_dir, class_name)
        if not os.path.exists(subset_class_dir):
            os.makedirs(subset_class_dir)
        images = os.listdir(class_dir)
        random.shuffle(images)
        for img in images[:num_images_per_class]:
            shutil.copy(os.path.join(class_dir, img), os.path.join(subset_class_dir, img))

# Get the intersection of class names in train, val, and test directories
train_classes = set(os.listdir(full_train_dir))
val_classes = set(os.listdir(full_val_dir))
test_classes = set(os.listdir(full_test_dir))

# Since classes are unique, we'll pick a fixed number from each split
classes_to_use_train = sorted(train_classes)[:50]  # First 50 classes from train
classes_to_use_val = sorted(val_classes)[:50]  # First 50 classes from val
classes_to_use_test = sorted(test_classes)[:50]  # First 50 classes from test

num_images_per_class = 500  # Increase to 500 images per class

# Create subsets independently for train, val, and test
create_subset(
    original_dir=full_train_dir,
    subset_dir=subset_train_dir,
    classes=classes_to_use_train,
    num_images_per_class=num_images_per_class
)

create_subset(
    original_dir=full_val_dir,
    subset_dir=subset_val_dir,
    classes=classes_to_use_val,
    num_images_per_class=num_images_per_class
)

create_subset(
    original_dir=full_test_dir,
    subset_dir=subset_test_dir,
    classes=classes_to_use_test,
    num_images_per_class=num_images_per_class
)

print("Subset creation complete.")


Subset creation complete.


### Section 5: Imports and Initial Setup

In [6]:
### Section 5: Imports and Initial Setup
!pip install keras-tuner

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import keras_tuner as kt
import os
import shutil
import random
import matplotlib.pyplot as plt
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.8 MB/s eta 0:00:00


### Section 6: Image Data Generators for the Subset (Updated for Data Augmentation)

---




In [7]:
# Section 6: Image data generators for the subset (Updated for More Aggressive Data Augmentation)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,  # Added vertical flip
    brightness_range=[0.8, 1.2],  # Added brightness adjustment
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


old version

### Section 7: Data Generators for the Subset


In [8]:
### Section 7: Data Generators for the Subset
train_generator = train_datagen.flow_from_directory(
    subset_train_dir,
    target_size=(84, 84),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    subset_val_dir,
    target_size=(84, 84),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    subset_test_dir,
    target_size=(84, 84),
    batch_size=32,
    class_mode='categorical'
)

Found 5000 images belonging to 50 classes.
Found 5000 images belonging to 50 classes.
Found 5000 images belonging to 50 classes.


### Section 8: Hyperparameter Tuning Function with Batch Size, Batch Normalization, and Regularization

In [9]:
### Section 8: Hyperparameter Tuning Function with Batch Size, Batch Normalization, and Regularization
from tensorflow.keras.layers import BatchNormalization  # Add this import

def build_model(hp):
    model = Sequential()

    # Print the batch size being used for this trial
    batch_size = hp.Int('batch_size', 16, 64, step=16)
    print(f"Batch size for this trial: {batch_size}")

    # First convolutional layer
    model.add(Conv2D(hp.Int('conv_1_filter', 32, 256, step=32), (3, 3), activation='relu', input_shape=(84, 84, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(BatchNormalization())

    # Second convolutional layer
    model.add(Conv2D(hp.Int('conv_2_filter', 64, 512, step=64), (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(BatchNormalization())

    # Third convolutional layer (new)
    model.add(Conv2D(hp.Int('conv_3_filter', 128, 512, step=64), (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(BatchNormalization())

    model.add(Flatten())

    # Dense layer with regularization
    model.add(Dense(hp.Int('dense_units', 64, 512, step=64), activation='relu',
                    kernel_regularizer=tf.keras.regularizers.l2(hp.Float('l2_regularization', 1e-4, 1e-2, sampling='LOG'))))
    model.add(Dropout(hp.Float('dropout_rate', 0.2, 0.5, step=0.1)))

    # Output layer
    model.add(Dense(train_generator.num_classes, activation='softmax'))

    # Compile the model with a tunable learning rate
    model.compile(optimizer=Adam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-2, sampling='LOG')),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


### Section 9: Early Stopping Callback

In [10]:
### Section 9: Callback for Early Stopping (Updated for longer patience)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,  # Increased patience for better fine-tuning
    restore_best_weights=True
)


### Section 10: Learning Rate Scheduler (New)

In [11]:
### Section 10: Learning Rate Scheduler
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=3,
    min_lr=1e-6,
    verbose=1
)


### Section 11: Tuner Setup (Including Batch Size Tuning)

In [ ]:
### Section 11: Tuner Setup (Corrected with Batch Size Tuning)
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='new_output',
    project_name='digit_tuning_subset'
)

# Start the hyperparameter search
tuner.search(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=[early_stopping, lr_scheduler]
)

# Log the hyperparameters, including batch size
tuner.search_space_summary()


Batch size for this trial: 16


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
48                |48                |batch_size
96                |96                |conv_1_filter
448               |448               |conv_2_filter
448               |448               |conv_3_filter
128               |128               |dense_units
0.0062534         |0.0062534         |l2_regularization
0.2               |0.2               |dropout_rate
0.00013322        |0.00013322        |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Batch size for this trial: 48
Epoch 1/2


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


157/157 ━━━━━━━━━━━━━━━━━━━━ 1162s 7s/step - accuracy: 0.0189 - loss: 6.5763 - val_accuracy: 0.0212 - val_loss: 5.4981 - learning_rate: 1.3322e-04
Epoch 2/2
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.0135 - loss: 6.1274

### Section 12: Train Model with Best Hyperparameters



In [ ]:
### Section 12: Train Model with Best Hyperparameters
print("Compiling and training the model with the best hyperparameters...")

# Build the best model from the hyperparameter search
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hps)

# Explicitly compile the model to ensure metrics are tracked
best_model.compile(
    optimizer=Adam(learning_rate=best_hps.get('learning_rate')),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model with the best hyperparameters, using the early stopping and learning rate scheduler callbacks
history = best_model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=[early_stopping, lr_scheduler]
)

# After training, print out the history keys to check for 'val_accuracy'
print("Available keys in history.history after training:", history.history.keys())

# Evaluate the model on the subset test data
test_loss, test_acc = best_model.evaluate(test_generator)
print(f"Test accuracy on subset: {test_acc}")


### Section 13: Cross-Validation Loop (Optional)


In [ ]:
### Section 13: Cross-Validation Loop (Optional)
from sklearn.model_selection import KFold

### Section 14. Cross-Validation Loop (Optional)
kf = KFold(n_splits=5)
cv_scores = []

for train_idx, val_idx in kf.split(train_data):
    train_data_fold = train_data[train_idx]
    val_data_fold = val_data[val_idx]

    history = best_model.fit(train_data_fold, epochs=10, validation_data=val_data_fold,
                             callbacks=[early_stopping, lr_scheduler])
    score = best_model.evaluate(val_data_fold)
    cv_scores.append(score)

print("Cross-validation scores:", cv_scores)


### Section 14: Get the Best Hyperparameters

In [ ]:
### Section 14: Get the Best Hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Print the best hyperparameters, including batch size
print("Best Hyperparameters:")
print(f"Batch size: {best_hps.get('batch_size')}")
print(f"conv_1_filter: {best_hps.get('conv_1_filter')}")
print(f"conv_2_filter: {best_hps.get('conv_2_filter')}")
print(f"conv_3_filter: {best_hps.get('conv_3_filter')}")
print(f"dense_units: {best_hps.get('dense_units')}")
print(f"l2_regularization: {best_hps.get('l2_regularization')}")
print(f"dropout_rate: {best_hps.get('dropout_rate')}")
print(f"learning_rate: {best_hps.get('learning_rate')}")



### Section 15: Save the results in a CSV file

In [ ]:
### Section 15: Save the results in a CSV file
results = []
for trial in tuner.oracle.get_best_trials(num_trials=10):
    trial_summary = {
        'trial_number': trial.trial_id,
        'conv_1_filter': trial.hyperparameters.get('conv_1_filter'),
        'conv_2_filter': trial.hyperparameters.get('conv_2_filter'),
        'dense_units': trial.hyperparameters.get('dense_units'),
        'dropout_rate': trial.hyperparameters.get('dropout_rate'),
        'learning_rate': trial.hyperparameters.get('learning_rate'),
        'batch_size': trial.hyperparameters.values['batch_size'] if 'batch_size' in trial.hyperparameters.values else 32,
        'accuracy': trial.metrics.get_last_value('accuracy'),
        'loss': trial.metrics.get_last_value('loss'),
        'val_accuracy': trial.metrics.get_last_value('val_accuracy'),
        'val_loss': trial.metrics.get_last_value('val_loss'),
        'test_accuracy': test_acc,
        'test_loss': test_loss,
    }
    results.append(trial_summary)

results_df = pd.DataFrame(results)
results_df.to_csv('new_output/trial_results.csv', index=False)


### Section 16: Train on Full Dataset (Updated)

In [ ]:
### Section 16: Train on Full Dataset (Updated)

# Use the best hyperparameters for the full dataset
batch_size = best_hps.values.get('batch_size', 32)  # Properly retrieve the batch size with a default fallback

train_generator_full = train_datagen.flow_from_directory(
    full_train_dir,
    target_size=(84, 84),
    batch_size=batch_size,  # Use the resolved batch_size
    class_mode='categorical'
)

val_generator_full = val_datagen.flow_from_directory(
    full_val_dir,
    target_size=(84, 84),
    batch_size=batch_size,  # Use the resolved batch_size
    class_mode='categorical'
)

test_generator_full = test_datagen.flow_from_directory(
    full_test_dir,
    target_size=(84, 84),
    batch_size=batch_size,  # Use the resolved batch_size
    class_mode='categorical'
)

# Build and train the model with the best hyperparameters on the full dataset
best_model_full = tuner.hypermodel.build(best_hps)

# Modify the final layer to match the number of classes in the full dataset
best_model_full.pop()  # Remove the old final layer
best_model_full.add(Dense(train_generator_full.num_classes, activation='softmax'))  # Add a new layer with correct output size

best_model_full.compile(optimizer=Adam(learning_rate=best_hps.get('learning_rate')),
                        loss='categorical_crossentropy',
                        metrics=['accuracy'])

history_full = best_model_full.fit(
    train_generator_full,
    epochs=20,
    validation_data=val_generator_full,
    callbacks=[early_stopping]
)

# Evaluate the model on the full test data
test_loss_full, test_acc_full = best_model_full.evaluate(test_generator_full)
print(f"Test accuracy on full dataset: {test_acc_full}")

# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_full.history['accuracy'], label='accuracy')
plt.plot(history_full.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_full.history['loss'], label='loss')
plt.plot(history_full.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

# Displaying the winning hyperparameters
best_hyperparameters_df = pd.DataFrame([{
    'conv_1_filter': best_hps.get('conv_1_filter'),
    'conv_2_filter': best_hps.get('conv_2_filter'),
    'dense_units': best_hps.get('dense_units'),
    'dropout_rate': best_hps.get('dropout_rate'),
    'learning_rate': best_hps.get('learning_rate'),
    'batch_size': batch_size  # Use the resolved batch size
}])
print(best_hyperparameters_df)
best_hyperparameters_df.to_csv('new_output/best_hyperparameters.csv', index=False)
